In [1]:
from statsbombpy import sb
import pandas as pd
import numpy as np

In [2]:
#All events sorted for barca home games and possession 
events = sb.competition_events(
    country="Germany",
    division= "1. Bundesliga",
    season="2023/2024",
    gender="male"
)

df = events[events["possession_team"]=="Bayer Leverkusen"]
df = df[df["team"]=="Bayer Leverkusen"]

/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarni

In [3]:
position_dict = {}
for id in events.match_id.unique():
    #find one match for one team
    match_subset = events.loc[events['match_id'] == id]

    #we identify the starting 11
    starting_11 = match_subset.loc[match_subset['type'] == 'Starting XI'].loc[match_subset['team'] == 'Bayer Leverkusen', 'tactics'].to_list()[0]
    starting_11_dict = {}

    
    #we make a dictionary for positions of players
    for member in starting_11['lineup']:
        player_id = member['player']['id']
        position_name = member['position']['name']
        starting_11_dict[player_id] = position_name
    
    position_dict[id] = {0 : starting_11_dict}

    match_subset = match_subset.loc[(match_subset['team'] == 'Bayer Leverkusen') & (match_subset['type'].isin(['Substitution', 'Tactical Shift']))]

    #sort the values like when we did the passing sequences
    match_subset = match_subset.sort_values(['period','timestamp'], ascending=[True, True])

    match_subset['pass_recipient_position'] = np.nan

    for index, row in match_subset.iterrows():
        #If substitution, we update the dictionary to include player
        if row['type'] == 'Substitution':
            match_dict = position_dict[id]
            latest_lineup = match_dict[list(match_dict.keys())[-1]]
            latest_lineup[row['substitution_replacement_id']] = row['position']
            position_dict[id][row['possession']] = latest_lineup
          
        #In case of a tactical shift, create a new position_dict
        if row['type'] == 'Tactical Shift':
            new_formation = row['tactics']
            lineup = {}
            for member in new_formation['lineup']:
                player_id = member['player']['id']
                position_name = member['position']['name']
                lineup[player_id] = position_name
            position_dict[id][row['possession']] = lineup

/var/folders/_3/q0p4rhwn0r31w83n5ds1htx40000gn/T/ipykernel_14108/3583605294.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  match_subset['pass_recipient_position'] = np.nan
/var/folders/_3/q0p4rhwn0r31w83n5ds1htx40000gn/T/ipykernel_14108/3583605294.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  match_subset['pass_recipient_position'] = np.nan
/var/folders/_3/q0p4rhwn0r31w83n5ds1htx40000gn/T/ipykernel_14108/3583605294.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fra

In [4]:
#filter threshold for Xg:
df_xg = df[~df['shot_statsbomb_xg'].between(0, 0.05)]

In [5]:
#Events sorted in a specific order so each passing sequence is correctly sorted
sequences_sorted = df_xg.sort_values(['match_id', 'period','timestamp'], ascending=[True, True, True])

In [6]:
#make new ids because right now there is ids from 1 to x for each match but it repeats from 1 and up in every match so each possession id points to different matches 
# - i just put the possession id after match_id in the newly created id
sequences_sorted['possession_id'] = sequences_sorted['match_id'].astype(str) + sequences_sorted['possession'].astype(str)
sequences_sorted['possession_id'] = sequences_sorted['possession_id'].astype(int)

/var/folders/_3/q0p4rhwn0r31w83n5ds1htx40000gn/T/ipykernel_14108/955078220.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sequences_sorted['possession_id'] = sequences_sorted['match_id'].astype(str) + sequences_sorted['possession'].astype(str)


In [7]:
#get the ids of sequences which contain a shot (contain an xg value)
shot_sequences = sequences_sorted[sequences_sorted["shot_statsbomb_xg"].notna()]
shot_sequences_ids = shot_sequences["possession_id"].unique()

In [8]:
#filter for possession sequences which end with a shot
sequences_filtered = sequences_sorted[sequences_sorted['possession_id'].isin(shot_sequences_ids)]

In [9]:
#fill all rows with an xg for the corresponding sequence - right now there are many missing values in "shot_statsbomb_xg"
sequences_filtered['xg'] = sequences_filtered.groupby('possession_id')['shot_statsbomb_xg'].transform(lambda group: group.fillna(method='ffill').fillna(method='bfill'))

/var/folders/_3/q0p4rhwn0r31w83n5ds1htx40000gn/T/ipykernel_14108/2851514471.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sequences_filtered['xg'] = sequences_filtered.groupby('possession_id')['shot_statsbomb_xg'].transform(lambda group: group.fillna(method='ffill').fillna(method='bfill'))
/var/folders/_3/q0p4rhwn0r31w83n5ds1htx40000gn/T/ipykernel_14108/2851514471.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sequences_filtered['xg'] = sequences_filtered.groupby('possession_id')['shot_statsbomb_xg'].transform(lambda group: group.fillna(method='ffill').fillna(method='bfill'))
/var/folders/_3/q0p4rhwn0r31w83n5ds1htx40000gn/T/ipykernel_14108/2851514471.py:2: Se

In [10]:
#find ids of those possessions that result in a goal
goal_posession_index = sequences_filtered.loc[sequences_filtered["shot_outcome"]=="Goal", "possession_id"]

In [11]:
#now we dont need the shot event rows any more so remove them
sequences_filtered = sequences_filtered[sequences_filtered["type"]!="Shot"]

Nu begynder jeg at arbejde på, at lave spillerne til nodes

In [12]:
#filter the df to only include row with an id the of a pass recipient and we subset the columns
player_final_sequences =  sequences_filtered[sequences_filtered["pass_recipient"].notna()][["match_id", "possession", "position", "pass_recipient_id", "possession_id", "xg"]]

In [13]:
#remove sequences with few passes if wanted
index_counts = player_final_sequences['possession_id'].value_counts()
player_final_sequences = player_final_sequences[player_final_sequences['possession_id'].isin(index_counts[index_counts > 5].index)]

In [14]:
#get all the unique possession ids for iteration
possession_index = player_final_sequences["possession_id"].unique()

In [15]:
player_final_sequences['pass_recipient_position'] = np.nan

for index, row in player_final_sequences.iterrows():
    recipient = row['pass_recipient_id']

    lineups = position_dict[row['match_id']]

    i = 0
    keys = list(lineups.keys())

    #Find out the current line up at the time of the event
    while i <= len(keys)-2 and row['possession'] > keys[i+1]:
        i += 1
    
    lineup = lineups[keys[i]]

    player_final_sequences.at[index, 'pass_recipient_position']  = lineup[recipient]


/var/folders/_3/q0p4rhwn0r31w83n5ds1htx40000gn/T/ipykernel_14108/3866115077.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Right Center Back' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  player_final_sequences.at[index, 'pass_recipient_position']  = lineup[recipient]


In [16]:
import networkx as nx

#iterate over possession ids and each row and append edges to a list for each graph and append that graph to a graphs dictionary (directed graph created with "nx.DiGraph(edges)")
#xg added as an attribute for each graph
graphs_dict = {}
for j in possession_index:
    edges = []
    for i in player_final_sequences.index:
        if j == player_final_sequences["possession_id"][i]:
            edges.append((player_final_sequences["position"][i], player_final_sequences["pass_recipient_position"][i]))
            if j not in graphs_dict:
                graphs_dict[j] = {"xg": player_final_sequences["xg"][i], "graph": None}
            else:
                graphs_dict[j]["xg"] = player_final_sequences["xg"][i]
                
    graph = nx.DiGraph(edges)
    graphs_dict[j]["graph"] = graph

graph_list = [value["graph"] for value in graphs_dict.values()]

{389505258: {'xg': 0.12706259,
  'graph': <networkx.classes.digraph.DiGraph at 0x7fd36aaae670>},
 389505264: {'xg': 0.13226183,
  'graph': <networkx.classes.digraph.DiGraph at 0x7fd378c1fee0>},
 3895052101: {'xg': 0.30528295,
  'graph': <networkx.classes.digraph.DiGraph at 0x7fd39b23b490>},
 3895052128: {'xg': 0.14435048,
  'graph': <networkx.classes.digraph.DiGraph at 0x7fd378c1fe50>},
 389506020: {'xg': 0.11212661,
  'graph': <networkx.classes.digraph.DiGraph at 0x7fd39b23b520>},
 389506036: {'xg': 0.21289383,
  'graph': <networkx.classes.digraph.DiGraph at 0x7fd39b23b700>},
 389506059: {'xg': 0.10086409,
  'graph': <networkx.classes.digraph.DiGraph at 0x7fd378c1fe80>},
 389506068: {'xg': 0.065433435,
  'graph': <networkx.classes.digraph.DiGraph at 0x7fd39b23b4f0>},
 389506085: {'xg': 0.20104417,
  'graph': <networkx.classes.digraph.DiGraph at 0x7fd38d338c10>},
 389506092: {'xg': 0.12446163,
  'graph': <networkx.classes.digraph.DiGraph at 0x7fd38d338bb0>},
 3895060119: {'xg': 0.06717

In [ ]:
edge_matrix = [list(graph.edges(data=True)) for graph in graph_list]
GRAPH_DB = graph_list  # List of graphs in the database
min_sup = 0

In [18]:
from itertools import combinations
from networkx.algorithms.isomorphism import DiGraphMatcher


def frequent_singletons(min_sup, edge_matrix):
    items_counted = {}
    edge_attributes = {}

    for edge_list in edge_matrix:
        for edge in edge_list:
            # Use only source and target nodes for counting
            edge_key = (edge[0], edge[1])
            items_counted[edge_key] = items_counted.get(edge_key, 0) + 1
            
            # Store the edge attributes
            if edge_key not in edge_attributes:
                edge_attributes[edge_key] = edge[2]

    # Filter edges that meet the min_sup
    F = [key for key, value in items_counted.items() if value >= min_sup]
    
    F_graphs = []
    for edge in F:
        g = nx.DiGraph()
        g.add_edge(edge[0], edge[1], **edge_attributes[edge])  # Add edge with original attributes
        F_graphs.append(g)
    
    return F_graphs

def generate_candidates(F, k):
    candidates = set()
    
    # Iterate over all pairs of frequent subgraphs (F)
    for g1, g2 in combinations(F, 2):

        edges_g1 = sorted(g1.edges(data=True), key=lambda e: e[2].get('sequence', 0))
        edges_g2 = sorted(g2.edges(data=True), key=lambda e: e[2].get('sequence', 0))
        edges_g1 = edges_g1[0]
        edges_g2 = edges_g2[0]
        
        if edges_g1[2].get('sequence') > edges_g2[2].get('sequence') or edges_g1[2].get('sequence') < edges_g2[2].get('sequence'):
            union_graph = nx.compose(g1, g2)
            
            # Ensure that the union has the correct number of edges
            if union_graph.number_of_edges() == k:  # Check edge size instead of node size
                candidates.add(union_graph)
    
    return candidates

# Count the support for each candidate in the graph database
def count_support(C, graph_db):
    F_count = {}
    for graph in graph_db:
        for candidate in C:
            GM = DiGraphMatcher(graph, candidate)
            if GM.subgraph_is_isomorphic():  # Check for subgraph isomorphism
                F_count[candidate] = F_count.get(candidate, 0) + 1
    return F_count

# Filter frequent candidates based on minimum support
def filter_frequent(F_count, min_sup):
    return [key for key, value in F_count.items() if value >= min_sup]

# Main function to run the apriori graph mining algorithm
def apriori_graph_mining(min_sup, edge_matrix, graph_db, max_k):
    frequent_total = []
    
    # Step 1: Find frequent singletons (edges)
    F = frequent_singletons(min_sup, edge_matrix)
    
    # Add initial frequent items to the total list
    frequent_total.extend(F)
    
    k = 2  # Start with size-2 subgraphs
    while k <= max_k:
        print(f"\nIteration {k}:")
        
        # Step 2: Generate candidate subgraphs of size k
        C = generate_candidates(F, k)
        
        # Step 3: Count support for each candidate in the graph database
        F_count = count_support(C, graph_db)
        
        # Step 4: Filter out frequent candidates that meet the minimum support
        F = filter_frequent(F_count, min_sup)
        
        if not F:  # If no frequent candidates are found, stop the algorithm
            print(f"No frequent subgraphs found for size {k}. Terminating.")
            break
        
        # Add frequent items to the total list
        frequent_total.extend(F)
        
        print(f"Frequent subgraphs of size {k}:")
        for subgraph in F:
            for u, v, attr in subgraph.edges(data=True):
                print(f"Edge {u} -> {v}, Attributes: {attr}")
        
        k += 1  # Move to the next size of subgraphs

    return frequent_total


graph_list = [value["graph"] for value in graphs_dict.values()]

edge_matrix = [list(graph.edges(data=True)) for graph in graph_list]
GRAPH_DB = graph_list  # List of graphs in the database
min_sup = 0


frequents = apriori_graph_mining(20, edge_matrix, GRAPH_DB, 5)
for pattern in frequents:
    print(pattern.edges())


Iteration 2:


TypeError: '>' not supported between instances of 'NoneType' and 'NoneType'